# Linear Regressions for house price prediction

In [ ]:
#Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import loguniform
from scipy.stats import uniform

from sklearn.datasets import fetch_california_housing
from sklearn.dummy import DummyRegressor

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_percentage_error

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline


# Common set up

In [ ]:
#set random seed
np.random.seed(306)

In [ ]:
# Configure shuffle splt
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

# Data Loading and splitting

In [ ]:
features, labels = fetch_california_housing(as_frame=True, return_X_y=True)
com_train_features, test_features, com_train_labels, test_labels = train_test_split(features,labels,random_state=42)
train_features, dev_features, train_labels, dev_labels = train_test_split(com_train_features, com_train_labels, random_state=42)

# Linear Regression with normal equation

In [ ]:
lin_reg_pipeline = Pipeline([('feature_scaling', StandardScaler()),
                             ('lin_reg',LinearRegression())])
lin_reg_cv_results = cross_validate(lin_reg_pipeline,
                                    com_train_features,
                                    com_train_labels,
                                    cv=cv,
                                    scoring='neg_mean_absolute_error',
                                    return_train_score=True,
                                    return_estimator=True)
lin_reg_train_error = -1 * lin_reg_cv_results['train_score']
lin_reg_test_error = -1 * lin_reg_cv_results['test_score']
print(lin_reg_train_error.mean(),'+/-',lin_reg_train_error.std())
print(lin_reg_test_error.mean(),'+/-',lin_reg_test_error.std())

# Linear regression with SGD

Let's use iterative optimization method to train linear regression model.

We use pipeline with two stages: 1. Feature Scaling and 2. SGD regression on the transformed feature matrix

In [ ]:
sgd_reg_pipeline = Pipeline([('feature_scaling', StandardScaler()),
                             ('sgd_reg',SGDRegressor(max_iter=np.ceil(1e6/com_train_features.shape[0]),
                                            early_stopping=True,
                                            eta0=1e-4,
                                            learning_rate='constant',
                                            tol=1e-5,
                                            validation_fraction=0.1,
                                            n_iter_no_change=5,
                                            average=10,
                                            random_state=42))])

sgd_reg_cv_results = cross_validation(sgd_reg_pipeline,
                              com_train_features,
                              com_train_labels,
                              cv=cv,
                              scoring='neg_mean_absolute_error',
                              return_train_score=True,
                              return_estimator=True)

sgd_train_error = -1 * sgd_reg_cv_results['train_score']
sgd_test_error = -1 * sgd_reg_cv_results['test_score']
print(sgd_train_error.mean(),'+/-',lin_reg_train_error.std())
print(sgd_reg_test_error.mean(),'+/-',lin_reg_test_error.std())

# Polynomial regression

We will use degree 2 followed by validation curve

In [ ]:
poly_reg_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                              ('feature_scaling', StandardScaler()),
                              ('lin_reg',LinearRegression())])
poly_reg_cv_results = cross_validate(poly_reg_pipeline,
                                    com_train_features,
                                    com_train_labels,
                                    cv=cv,
                                    scoring='neg_mean_absolute_error',
                                    return_train_score=True,
                                    return_estimator=True)
poly_reg_train_error = -1 * poly_reg_cv_results['train_score']
poly_reg_test_error = -1 * poly_reg_cv_results['test_score']
print(poly_reg_train_error.mean(),'+/-',poly_reg_train_error.std())
print(poly_reg_test_error.mean(),'+/-',poly_reg_test_error.std())

Notice the reduction in error

We would now use interaction feature terms only

In [ ]:
poly_reg_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2, interaction_only=True)),
                              ('feature_scaling', StandardScaler()),
                              ('lin_reg',LinearRegression())])
poly_reg_cv_results = cross_validate(poly_reg_pipeline,
                                    com_train_features,
                                    com_train_labels,
                                    cv=cv,
                                    scoring='neg_mean_absolute_error',
                                    return_train_score=True,
                                    return_estimator=True)
poly_reg_train_error = -1 * poly_reg_cv_results['train_score']
poly_reg_test_error = -1 * poly_reg_cv_results['test_score']
print(poly_reg_train_error.mean(),'+/-',poly_reg_train_error.std())
print(poly_reg_test_error.mean(),'+/-',poly_reg_test_error.std())

Let's figure out the best degree suited for regression task

In [ ]:
degree = [1,2,3,4,5]

train_scores, test_scores = validation_curve(poly_reg_pipeline, com_train_features, com_train_labels, param_name='poly__degree',
                                             param_range=degree, cv=cv, scoring='neg_mean_absolute_error', n_jobs=2)

train_errors, test_errors = -train_scores, -test_scores
plt.plot(degree, train_errors.mean(axis=1),'b-x',label='Training error')
plt.plot(degree, test_errors.mean(axis=1),'b-x',label='Test error')
plt.legend()

plt.ylabel('degree')
plt.xlabel('Mean absolute error (k$)')
_=plt.title('Validation curve for polynomial regression')

#Ridge Regression

The polynomial models have a tendency to overfit- if we use higher order polynomial features. We will use Ridge regression-which penalizes for excessive model complexity in the polynomial regression by adding a regularization term. Here we specify the regularization rate alpha as 0.5 and train the regression model. Later we will launch hyperparameter search for the right value of alpha such that it leads to the least cross validation errors.

In [ ]:
ridge_reg_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                              ('feature_scaling', StandardScaler()),
                              ('ridge',Ridge(alpha=0.5))])
ridge_reg_cv_results = cross_validate(ridge_reg_pipeline,
                                      com_train_features,
                                      com_train_labels,
                                      cv=cv,
                                      scoring='neg_mean_absolute_error',
                                      return_train_score=True,
                                      return_estimator=True)
ridge_reg_train_error = -1 * ridge_reg_cv_results['train_score']
ridge_reg_test_error = -1 * ridge_reg_cv_results['test_score']
print(ridge_reg_train_error.mean(),'+/-',ridge_reg_train_error.std())
print(ridge_reg_test_error.mean(),'+/-',ridge_reg_test_error.std())

# HPT for ridge regularization rate

In [ ]:
alpha_list = np.logspace(-4,0,num=20)
ridge_reg_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                              ('feature_scaling', StandardScaler()),
                              ('ridge_cv',RidgeCV(alpha=alpha_list,
                                                  cv=cv,
                                                  scoring='neg_mean_absolute_error'))])
ridge_reg_cv_results = ridge_reg_pipeline.fit(com_train_features, com_train_labels)

In [ ]:
print('Score with best alpha', ridge_reg_cv_results[-1].best_score_)
print('Error with best alpha', -ridge_reg_cv_results[-1].best_score_)
print('Best alpha', ridge_reg_cv_results[-1].alpha_)

# Ridge HPT through GridSearchCV

In [ ]:
ridge_grid_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                                ('feature_scaling', StandardScaler()),
                                ('ridge',Ridge())])

param_grid = {'poly__degree':(1,2,3),
              'ridge__alpha':np.logspace(-4,0,num=20)}

ridge_grid_search = GridSearchCV(ridge_grid_pipeline,
                                 param_grid=param_grid,
                                 n_jobs=2,
                                 cv=cv,
                                 scoring='neg_mean_absolute_error',
                                 return_train_score=True)
ridge_grid_search.fit(com_train_features, com_train_labels)

ridge_ridge_search.best_index_ gives us the index of the best parameter in the list.

In [ ]:
mean_train_error = -1 * ridge_grid_search.cv_results_['mean_train_score'][ridge_ridge_search.best_index_]
mean_test_error = -1 * ridge_grid_search.cv_results_['mean_test_score'][ridge_ridge_search.best_index_]
std_train_error = ridge_grid_search.cv_results_['std_train_score'][ridge_ridge_search.best_index_]
std_test_error = ridge_grid_search.cv_results_['std_test_score'][ridge_ridge_search.best_index_]

In [ ]:
print('Best mean absolute error of polynomial ridge regression model on the train set:', mean_train_error,' +/- ', std_train_error)
print('Mean absolute error of polynomial ridge regression model on the test set:', mean_test_error,' +/- ', std_test_error)

In [ ]:
print('Mean cross validated score', ridge_grid_search.best_score_)
print('Mean cross validated error', -ridge_grid_search.best_score_)
print('Best alpha', ridge_grid_search.best_params_)

# Lasso regression

## Baseline model with fixed learning rate

In [ ]:
lasso_reg_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                              ('feature_scaling', StandardScaler()),
                              ('lasso',lasso(alpha=0.01))])
lasso_reg_cv_results = cross_validate(lasso_reg_pipeline,
                                      com_train_features,
                                      com_train_labels,
                                      cv=cv,
                                      scoring='neg_mean_absolute_error',
                                      return_train_score=True,
                                      return_estimator=True)
lasso_reg_train_error = -1 * lasso_reg_cv_results['train_score']
lasso_reg_test_error = -1 * lasso_reg_cv_results['test_score']
print(lasso_reg_train_error.mean(),'+/-',lasso_reg_train_error.std())
print(lasso_reg_test_error.mean(),'+/-',lasso_reg_test_error.std())

# HPT for lasso regularization rate

with GridSearchCV

In [ ]:
lasso_grid_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                              ('feature_scaling', StandardScaler()),
                              ('lasso',Lasso())])

param_grid = {'poly__degree':(1,2,3),
              'lasso_alpha': np.logspace(-4,0,num=20)}

lasso_grid_search = GridSearchCV(lasso_grid_pipeline,
                                 param_grid = param_grid,
                                 n_jobs=2,
                                 cv=cv,
                                 scoring='neg_mean_absolute_error',
                                 return_train_score=True)
lasso_grid_search.fit(com_train_features, com_train_labels)

In [ ]:
mean_train_error = -1 * lasso_grid_search.cv_results_['mean_train_score'][lasso_grid_search.best_index_]
mean_test_error = -1 * lasso_grid_search.cv_results_['mean_test_score'][lasso_grid_search.best_index_]
std_train_error = lasso_grid_search.cv_results_['std_train_score'][lasso_grid_search.best_index_]
std_test_error = lasso_grid_search.cv_results_['std_test_score'][lasso_grid_search.best_index_]

In [ ]:
print('Best mean absolute error of polynomial lasso regression model on the train set:', mean_train_error,' +/- ', std_train_error)
print('Mean absolute error of polynomial lasso regression model on the test set:', mean_test_error,' +/- ', std_test_error)

In [ ]:
print('Mean cross validated score', lasso_grid_search.best_score_)
print('Mean cross validated error', -lasso_grid_search.best_score_)
print('Best alpha', lasso_grid_search.best_params_)

# SGD: Regularization and HPT

In [ ]:
poly_sgd_pipeline = Pipeline([('poly', PolynomialFeatures(degree=2)),
                              ('feature_scaling', StandardScaler()),
                              ('sgd_reg',SGDRegressor(penalty='elasticnet',
                                                      random_state=42))])
poly_sgd_cv_results = cross_validate(poly_sgd_pipeline,
                                      com_train_features,
                                      com_train_labels,
                                      cv=cv,
                                      scoring='neg_mean_absolute_error',
                                      return_train_score=True,
                                      return_estimator=True)
poly_sgd_train_error = -1 * poly_sgd_cv_results['train_score']
poly_sgd_test_error = -1 * poly_sgd_cv_results['test_score']
print(poly_sgd_train_error.mean(),'+/-',poly_sgd_train_error.std())
print(poly_sgd_test_error.mean(),'+/-',poly_sgd_test_error.std())

Let's search for best set of params for ploy+SGD

In [ ]:
class uniform_int:
  def __init__(self,a,b):
    self.distribution = uniform(a,b)
  def rvs(self, *args, **kwargs):
    return self._distribution.rvs(*args, **kwargs).astype(int)

Let's specify RandomizedSearchCV setup

In [ ]:
param_distributions = {
    'poly__degree':[1,2,3],
    'sgd_reg__learning_rate':['constant','adaptive','invscalig'],
    'sgd_reg__l1_ratio':uniform(0,1),
    'sgd_reg__eta0':loguniform(1e-5,1),
    'sgd_reg__power_t':uniform(0,1)
}

poly_sgd_random_search = RandomizedSearchCV(
    poly_sgd_pipeline, param_dsitributions=param_distributions,
    n_iter=10,cv=cv,verbose=1,scoring='neg_mean_absolute_error'
)

poly_sgd_random_search.fit(com_train_features, com_train_labels)

The best score can be obtained as follows:

In [ ]:
poly_sgd_random_search.best_score_

The best set of parameters are obtained as follows:

In [ ]:
poly_sgd_random_search.best_params_

and the best estimator can be accessed with best_estimator_ member variable.

# Comparison of weight vectors

Let's look at the weight vectors produced by different models.

In [ ]:
feature_names = poly_reg_cv_results['estimator'][0][0].get_feature_names_out(
    input_features=train_features.columns)
feature_names

In [ ]:
coefs = [est[-1].coef_ for est in poly_reg_cv_results[estimator]]
weights_polynomial_regression = pd.DataFrame(coefs, columns=feature_names)

In [ ]:
color = {'whiskers':'black','medians':'black','caps':'black'}
weights_polynomial_regression.plot.box(color=color,vert=False,figsize=(6,16))
_=plt.title('Polynomial regression coefficients')

# Performance on the test set

Baseline

In [ ]:
baseline_model_median = DummyRegressor(strategy='median')
baseline_model_median.fit(train_features, train_labels)
mean_absolute_percentage_error(test_labels,baseline_model_median.predict(test_features))

# Linear regression with normal equation

In [ ]:
mean_absolute_percentage_error(test_labels,lin_reg_cv_results['estimator'][0][0].predict(test_features))

Polynomial regression

In [ ]:
mean_absolute_percentage_error(test_labels,poly_sgd_random_search.best_estimator_.predict(test_features))

Ridge Regression

In [ ]:
mean_absolute_percentage_error(test_labels,ridge_grid_search.best_estimator_.predict(test_features))

Lasso Regression

In [ ]:
mean_absolute_percentage_error(test_labels,lasso_grid_search.best_estimator_.predict(test_features))